# Masked Language Modeling

In [24]:
from transformers import pipeline, logging
logging.set_verbosity_error()

mask_filler = pipeline("fill-mask", model="roberta-base")

input_sentence = "Hanoi is the <mask> of Vietnam."

predictions = mask_filler(input_sentence, top_k=5)

print(f"Câu gốc: {input_sentence}")
for pred in predictions:
    print(f"\nDự đoán: {pred["token_str"]}; với độ tin cậy: {pred["score"]:.4f}")
    print(f"-> Câu hoàn chỉnh: {pred["sequence"]}")


Câu gốc: Hanoi is the <mask> of Vietnam.

Dự đoán:  capital; với độ tin cậy: 0.9934
-> Câu hoàn chỉnh: Hanoi is the capital of Vietnam.

Dự đoán:  Capital; với độ tin cậy: 0.0054
-> Câu hoàn chỉnh: Hanoi is the Capital of Vietnam.

Dự đoán:  Republic; với độ tin cậy: 0.0004
-> Câu hoàn chỉnh: Hanoi is the Republic of Vietnam.

Dự đoán:  center; với độ tin cậy: 0.0003
-> Câu hoàn chỉnh: Hanoi is the center of Vietnam.

Dự đoán:  heart; với độ tin cậy: 0.0001
-> Câu hoàn chỉnh: Hanoi is the heart of Vietnam.


- Mô hình đã dự đoán đúng từ "capital" với xác xuất 99%
- Các mô hình Encoder-only như này phù hợp với tác vụ này vì:

  - Được huấn luyện trực tiếp bằng Masked Language Modeling

  - Có self-attention hai chiều (bidirectional) -> tận dụng toàn bộ ngữ cảnh


# Next Token Prediction

In [25]:
from transformers import pipeline, logging
logging.set_verbosity_error()

generator = pipeline("text-generation", model="gpt2-medium")

prompt = "The best thing about learning NLP is"

generated_texts = generator(prompt,
                            max_new_tokens=50,
                            num_return_sequences=1,
                            pad_token_id=generator.tokenizer.eos_token_id)

print(f"Câu mồi: '{prompt}'")
for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text["generated_text"])

Câu mồi: 'The best thing about learning NLP is'
Văn bản được sinh ra:
The best thing about learning NLP is that it's not hard. The only thing you need to know is the right questions and the right answer. Your intuition and your understanding of your environment will help you figure out what's going on.


- Kết quả sinh ra hợp lý
- Các mô hình Decoder-only như này phù hợp với tác vụ này vì:
  - Được huấn luyện bằng causal language modeling
  - Self-attention chỉ nhìn trái, phù hợp cho từ tương lai chưa biết
  - Có cơ chế sinh nội dung auto-regressive

# Sentence Representation

In [26]:
import torch
from transformers import AutoTokenizer, AutoModel, logging
logging.set_verbosity_error()

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

sentences = ["This is a sample sentence."]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_state = outputs.last_hidden_state

attention_mask = inputs["attention_mask"]
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
sentence_embedding = sum_embeddings / sum_mask

print("Vector biểu diễn câu:")
print(sentence_embedding)
print("\nKích thước của vector:", sentence_embedding.shape)


Vector biểu diễn câu:
tensor([[-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5784e-02,
         -2.1826e-01,  4.7636e-01,  4.8659e-01,  4.0647e-05, -7.4273e-02,
         -7.4740e-02, -4.7635e-01, -1.9773e-01,  2.4824e-01, -1.2162e-01,
          1.6678e-01,  2.1045e-01, -1.4576e-01,  1.2636e-01,  1.8635e-02,
          2.4640e-01,  5.7090e-01, -4.7014e-01,  1.3782e-01,  7.3650e-01,
         -3.3808e-01, -5.0331e-02, -1.6452e-01, -4.3517e-01, -1.2900e-01,
          1.6516e-01,  3.4004e-01, -1.4930e-01,  2.2422e-02, -1.0488e-01,
         -5.1916e-01,  3.2964e-01, -2.2162e-01, -3.4206e-01,  1.1993e-01,
         -7.0148e-01, -2.3126e-01,  1.1224e-01,  1.2550e-01, -2.5191e-01,
         -4.6374e-01, -2.7261e-02, -2.8415e-01, -9.9249e-02, -3.7017e-02,
         -8.9192e-01,  2.5005e-01,  1.5816e-01,  2.2701e-01, -2.8497e-01,
          4.5300e-01,  5.0945e-03, -7.9441e-01, -3.1008e-01, -1.7403e-01,
          4.3029e-01,  1.6816e-01,  1.0590e-01, -4.8987e-01,  3.1856e-01,
          3.2861

- Kích thước của vector biểu diễn là 768. Con số này tương ứng với tham số hidden_size của mô hình BERT
- Cần sử dụng attention_mask để loại bỏ token padding khi mean pooling để embedding không bị sai lệch
